$$
EmotiWand = Sentimento^{\alpha}*Momentum^{\beta}*Volume^{\gamma}*Volatilidade^{\delta}*Trends^{\theta}
$$
Substituir EmotiWand por ibiv_close
$$
ln(HotSpots) = {\alpha}*ln(Sentimentos)+{\beta}*ln(Momentum)+{\gamma}*ln(Volume)+{\delta}*ln(Volatilidade)+{\theta}*ln(Trends)
$$

- Sentimento deveria ser estimado em ln?
- ibov_close é a melhor série pra um "greed and fear"?

Séries de tempo podem ter cointegração --> teste de Engle-Granger, Johansen e/ou ADL/Correção de erros --> estimar parâmetros

**Ideia**: selecionar períodos aleatórios de rebalanceamento : série binária com 1=(data teve rebalanceamento); repetir várias vezes -->    
Pegar as séries com melhores resultados --> rodar a regressão nessas séries e fazer média dos parâmetros ponderados por (1-pvalor)  
Resultado: índice com melhores momentos de rebalanceamento

**Ideia**: na hora de simular as carteiras com rebalançeamento em momentos aleatórios, a quantidade de rebalanceamentos (frequência) vai ser dada por uma distribuição (qui-quadrado? poisson?) --> depois, a gente vê a distribuição da frequência das carteiras mais eficientes pra saber apx a frequência ótima de rebalanceamentos --> depois roda com essa nova distribuição mais eficiente

In [29]:
import pandas as pd
import numpy as np
import os
import sqlite3
import matplotlib.pyplot as plt

from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [30]:
os.chdir(os.getcwd())
conn = sqlite3.connect(r"C:\databases\portfolio_quantamental.db")
cur = conn.cursor()

ibov_close = pd.read_sql('SELECT * FROM ibov_adj_close', conn)
ibov_close = ibov_close[(ibov_close['Date'] >= '2011-01-01') & (ibov_close['Date'] <= '2020-12-31')]

ibov_vol = pd.read_sql('SELECT * FROM ibov_volume', conn)
ibov_vol = ibov_vol[(ibov_vol['Date'] >= '2011-01-01') & (ibov_vol['Date'] <= '2020-12-31')]

vix = pd.read_excel(r'C:\Users\loren\Documents\Programação\Github\quantamental-Dev_Wizards\data\vix\PerformanceGraphExport.xls')
vol = pd.read_excel(r'C:\Users\loren\Documents\Programação\Github\quantamental-Dev_Wizards\data\vix\IVol-BR.xls')
sent = pd.read_csv(r'C:\Users\loren\Documents\Programação\Github\quantamental-Dev_Wizards\data\news\sentimentos.csv')
trends = pd.read_csv(r'C:\Users\loren\Documents\Programação\Github\quantamental-Dev_Wizards\data\trends\interesses_google.csv')

In [31]:
vix.rename(columns={'Effective date ': 'Date'}, inplace=True)
sent.rename(columns={'Data': 'Date'}, inplace=True)
trends.rename(columns={'date': 'Date'}, inplace=True)

vol['Date'] = pd.to_datetime(vol[['year', 'month', 'day']])
vol.drop(columns=['year', 'month', 'day'], inplace=True)

In [32]:
# Ensure the 'Date' columns are in datetime format
ibov_close['Date'] = pd.to_datetime(ibov_close['Date'])
ibov_vol['Date'] = pd.to_datetime(ibov_vol['Date'])
vix['Date'] = pd.to_datetime(vix['Date'])
vol['Date'] = pd.to_datetime(vol['Date'])
sent['Date'] = pd.to_datetime(sent['Date'])
trends['Date'] = pd.to_datetime(trends['Date'])

In [33]:
start = sent['Date'].min().strftime('%Y-%m-%d')
end = sent['Date'].max().strftime('%Y-%m-%d')
print(f'{start} - {end}')

2014-09-14 - 2024-09-08


In [34]:
print(len(ibov_close), len(ibov_vol), len(vix), len(vol), len(sent))   

2481 2481 749 2415 522


In [36]:
# Merge the dataframes on the date columns
merged_df = ibov_close[['Date', 'BBAS3.SA']].merge(
    ibov_vol[['Date', 'BBAS3.SA']], on='Date', suffixes=('_close', '_vol'), how='inner'
).merge(
    vix[['Date', 'S&P/B3 Ibovespa VIX']], on='Date', how='inner'
).merge(
    vol[['Date', 'ivolbr']], on='Date', how='inner'
).merge(
    sent[['Date', 'Signal']], on='Date', how='inner'
)

# Rename columns for clarity
merged_df.rename(columns={
    'BBAS3.SA_close': 'ibov_close',
    'BBAS3.SA_vol': 'ibov_vol',
    'S&P/B3 Ibovespa VIX': 'vix',
    'ivolbr': 'vol',
    'Signal': 'sent_score'
}, inplace=True)

print(merged_df.head())

Empty DataFrame
Columns: [Date, ibov_close, ibov_vol, vix, vol, sent_score]
Index: []
